# Compile with the following OPTIONS File:

*# Options for compiling*<br>
**SHELL = /bin/bash**

*# Choose the COMPILER*<br>
**COMPILER = ifort**

*# Choose the Linear Algebra libraries*<br>
**LINALG = MKL**

*# Do you use DEBUG or Optimization Mode? (DEBUG/OPT)*<br>
**MODE = OPT**

*# Turn on omp or mpi (enter TRUE .or. FALSE)?*<br>
**OPENMP = FALSE**<br>
**MPI = FALSE**

*# Choose the MACHINE file*<br>
**MACHINE = HKQAI**

*# Choose HORSFIELD or McWEDA*<br>
**THEORY = HORSFIELD**

*# Choose HARRIS, DOGS, or KS (Kohn-Sham)*<br>
*# Note that the KS code only does LDA for now as it uses McWEDA routines.*<br> 
**SCF = DOGS**

*# Choose the DIAGONALIZATION Method - GAMMAD (divide and conquer),*<br>
*# KPOINTS, or MDET (also GAMMA) - Default is GAMMA*<br>
**DIAGONALIZATION = GAMMAD**

*# Choose the XC functional*<br>
**XC = BLYP**


# Preparation

In [1]:
# link the executable
!rm fireball.x
!ln -s ~/thunder2022/fireball/src/fireball.x

In [2]:
# link the Fdata
!rm Fdata
!ln -s ~/thunder2022/datafiles/Fdata-Horsfield-0.10-9SN.Hs4.10-9DN.Cs4.35p4.80.Ns3.95p4.40.Os3.35p3.80 Fdata

In [3]:
from thunder_ase.fireball import Fireball
import numpy as np
import ase
import sys
from ase.build import molecule
from ase import units

# Socket Communication
from ase.calculators.socketio import SocketIOCalculator

# ase BFGS optimization
from ase.optimize import BFGS

# ase MD modules
from ase.md.verlet import VelocityVerlet as NVE
from ase.md.nvtberendsen import NVTBerendsen

## Basic Setup

In [4]:
# Sockets
unixsocket = 'ase_server_socket'

# Read methane structure
atoms = ase.io.read('methane.xyz')

## Example 1: Optimization

In [5]:
# clean last run result
!./clean.com

rm: cannot remove 'Fdata.inp': No such file or directory
rm: cannot remove 'structures.inp': No such file or directory
rm: cannot remove 'structures.options': No such file or directory
rm: cannot remove 'structures.output': No such file or directory
rm: cannot remove 'structures.xsfoptions': No such file or directory
rm: cannot remove '*.CHARGES': No such file or directory
rm: cannot remove '*.eigen': No such file or directory
rm: cannot remove '001.inp': No such file or directory
rm: cannot remove '001.xyz': No such file or directory
rm: cannot remove '*.json': No such file or directory
rm: cannot remove 'WARNINGS': No such file or directory


In [6]:
dyn = BFGS(atoms, trajectory='opt.traj', logfile='opt.log')

kwargs = {
          'ipi': 1,
          'iwriteout_charges': 1,
          'max_scf_iterations_set': 100,
          'scf_tolerance_set': 0.00000001
          }
fireball = Fireball(command='./fireball.x', Fdata_path='Fdata', **kwargs)

with SocketIOCalculator(fireball, log=sys.stdout,
                        unixsocket=unixsocket) as calc:
    atoms.set_calculator(calc)
    dyn.run(fmax=0.05)  # run until maximum force is 0.05 eV/A

Accepting clients on UNIX-socket /tmp/ipi_ase_server_socket
Close socket server


TypeError: replace() argument 2 must be str, not None

## Example 2: NVE Ensemble

In [16]:
# clean last run result
!./clean.com

rm: cannot remove 'methane.inp': No such file or directory
rm: cannot remove '*.ENERGIES': No such file or directory
rm: cannot remove '*.denmat': No such file or directory
rm: cannot remove '*.dat': No such file or directory
rm: cannot remove 'WARNINGS': No such file or directory


In [17]:
kwargs = {
          'iwriteout_charges': 1,
          'max_scf_iterations_set': 100,
          'scf_tolerance_set': 0.00000001
          }

calc = Fireball(command='./fireball.x', 
                Fdata_path='Fdata',
                **kwargs)
atoms.set_calculator(calc)
atoms.rattle(stdev=0.1)

dyn = NVE(atoms, timestep=0.25 * units.fs, trajectory='md.traj', logfile='md.log')
dyn.run(1000)  # run 1000 steps 

FIREBALL RUNTIME :      2.04 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.46 [sec]  
FIREBALL RUNTIME :      2.16 [sec]  
FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      2.04 [sec]  
FIREBALL RUNTIME :      2.42 [sec]  
FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      1.83 [sec]  
FIREBALL RUNTIME :      2.42 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.16 [sec]  
FIREBALL RUNTIME :      1.81 [sec]  
FIREBALL RUNTIME :      1.82 [sec]  
FIREBALL RUNTIME :      2.06 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.27 [sec]  
FIREBALL RUNTIME :      2.27 [sec]  
FIREBALL RUNTIME :      2.30 [sec]  
FIREBALL RUNTIME :      2.38 [sec]  
FIREBALL RUNTIME :      2.04 [sec]  
FIREBALL RUNTIME :      2.35 [sec]  
FIREBALL RUNTIME :      2.45 [sec]  
FIREBALL RUNTIME :      1.96 [sec]  
FIREBALL RUNTIME :      2.01 [sec]  
FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      1.82 [sec]  
F

FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.45 [sec]  
FIREBALL RUNTIME :      2.37 [sec]  
FIREBALL RUNTIME :      2.38 [sec]  
FIREBALL RUNTIME :      2.46 [sec]  
FIREBALL RUNTIME :      2.32 [sec]  
FIREBALL RUNTIME :      1.78 [sec]  
FIREBALL RUNTIME :      1.78 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.43 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.22 [sec]  
FIREBALL RUNTIME :      2.36 [sec]  
FIREBALL RUNTIME :      2.05 [sec]  
FIREBALL RUNTIME :      1.78 [sec]  
FIREBALL RUNTIME :      1.77 [sec]  
FIREBALL RUNTIME :      2.26 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.44 [sec]  
FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.10 [sec]  
FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      2.45 [sec]  
F

FIREBALL RUNTIME :      2.37 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.30 [sec]  
FIREBALL RUNTIME :      2.31 [sec]  
FIREBALL RUNTIME :      2.17 [sec]  
FIREBALL RUNTIME :      2.30 [sec]  
FIREBALL RUNTIME :      1.79 [sec]  
FIREBALL RUNTIME :      2.44 [sec]  
FIREBALL RUNTIME :      1.82 [sec]  
FIREBALL RUNTIME :      2.30 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      1.98 [sec]  
FIREBALL RUNTIME :      1.79 [sec]  
FIREBALL RUNTIME :      2.43 [sec]  
FIREBALL RUNTIME :      2.13 [sec]  
FIREBALL RUNTIME :      2.40 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.34 [sec]  
FIREBALL RUNTIME :      2.41 [sec]  
FIREBALL RUNTIME :      2.13 [sec]  
FIREBALL RUNTIME :      2.43 [sec]  
FIREBALL RUNTIME :      2.27 [sec]  
FIREBALL RUNTIME :      2.39 [sec]  
FIREBALL RUNTIME :      2.38 [sec]  
FIREBALL RUNTIME :      2.23 [sec]  
FIREBALL RUNTIME :      2.33 [sec]  
F

forrtl: error (69): process interrupted (SIGINT)
Image              PC                Routine            Line        Source             
fireball.x         00000000017437FB  Unknown               Unknown  Unknown
libpthread-2.31.s  00007FE9EF025420  Unknown               Unknown  Unknown
fireball.x         00000000017B8F61  Unknown               Unknown  Unknown
fireball.x         0000000001769368  Unknown               Unknown  Unknown
fireball.x         0000000001767BBE  Unknown               Unknown  Unknown
fireball.x         00000000005137C1  Unknown               Unknown  Unknown
fireball.x         0000000000403FBB  Unknown               Unknown  Unknown
fireball.x         0000000000403A22  Unknown               Unknown  Unknown
libc-2.31.so       00007FE9EECF4083  __libc_start_main     Unknown  Unknown
fireball.x         000000000040392E  Unknown               Unknown  Unknown


KeyboardInterrupt: 

## Example 3: NVT Ensemble

In [9]:
# clean last run result
!./clean.com

/bin/bash: ./clean.sh: No such file or directory


In [10]:
kwargs = {
          'iwriteout_charges': 1,
          'max_scf_iterations_set': 100,
          'scf_tolerance_set': 0.00000001
          }

calc = Fireball(command='fireball.x', 
                Fdata_path='Fdata',
                **kwargs)
atoms.set_calculator(calc)
atoms.rattle(stdev=0.1)

dyn = NVE(atoms, timestep=0.25 * units.fs, trajectory='md.traj', logfile='md.log')
dyn.run(1000)  # run 1000 steps

NameError: name 'Fdata_path' is not defined